In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
Neo4j_Pass = os.environ['NEO4J_PASS']

In [1]:
from neo4j import GraphDatabase

class CRMNeo4j:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
    
    def add_customer(self, customer_id, name, email):
        with self.driver.session() as session:
            session.write_transaction(self._create_customer, customer_id, name, email)

    def add_product(self, product_id, name, price):
        with self.driver.session() as session:
            session.write_transaction(self._create_product, product_id, name, price)

    def add_purchase(self, customer_id, product_id, date):
        with self.driver.session() as session:
            session.write_transaction(self._create_purchase, customer_id, product_id, date)

    @staticmethod
    def _create_customer(tx, customer_id, name, email):
        query = (
            "CREATE (c:Customer {customer_id: $customer_id, name: $name, email: $email}) "
            "RETURN c"
        )
        tx.run(query, customer_id=customer_id, name=name, email=email)

    @staticmethod
    def _create_product(tx, product_id, name, price):
        query = (
            "CREATE (p:Product {product_id: $product_id, name: $name, price: $price}) "
            "RETURN p"
        )
        tx.run(query, product_id=product_id, name=name, price=price)

    @staticmethod
    def _create_purchase(tx, customer_id, product_id, date):
        query = (
            "MATCH (c:Customer {customer_id: $customer_id}), (p:Product {product_id: $product_id}) "
            "CREATE (c)-[:PURCHASED {date: $date}]->(p) "
            "RETURN c, p"
        )
        tx.run(query, customer_id=customer_id, product_id=product_id, date=date)

    def get_customer_purchases(self, customer_id):
        with self.driver.session() as session:
            return session.read_transaction(self._get_customer_purchases, customer_id)

    @staticmethod
    def _get_customer_purchases(tx, customer_id):
        query = (
            "MATCH (c:Customer {customer_id: $customer_id})-[:PURCHASED]->(p:Product) "
            "RETURN p.product_id AS product_id, p.name AS name, p.price AS price"
        )
        result = tx.run(query, customer_id=customer_id)
        return [record.data() for record in result]

    def get_product_customers(self, product_id):
        with self.driver.session() as session:
            return session.read_transaction(self._get_product_customers, product_id)

    @staticmethod
    def _get_product_customers(tx, product_id):
        query = (
            "MATCH (p:Product {product_id: $product_id})<-[:PURCHASED]-(c:Customer) "
            "RETURN c.customer_id AS customer_id, c.name AS name, c.email AS email"
        )
        result = tx.run(query, product_id=product_id)
        return [record.data() for record in result]

In [3]:
# Initialize the CRM system and connect to Neo4j
crm = CRMNeo4j("bolt://localhost:7687", "neo4j", Neo4j_Pass)

# Add customers
crm.add_customer("CUST001", "John Doe", "john@example.com")
crm.add_customer("CUST002", "Jane Smith", "jane@example.com")

# Add products
crm.add_product("PROD001", "Laptop", 999.99)
crm.add_product("PROD002", "Smartphone", 499.99)

# Add purchases
crm.add_purchase("CUST001", "PROD001", "2024-01-10")
crm.add_purchase("CUST002", "PROD002", "2024-01-15")

# Retrieve customer purchases
purchases = crm.get_customer_purchases("CUST001")
print(f"John's Purchases: {purchases}")

# Retrieve product's customers
customers = crm.get_product_customers("PROD002")
print(f"Smartphone Customers: {customers}")

# Close connection
crm.close()

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_1204/2477979760.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_customer, customer_id, name, email)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_1204/2477979760.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_product, product_id, name, price)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_1204/2477979760.py:21: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_purchase, customer_id, product_id, date)
/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_1204/2477979760.py:50: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(self._get_customer_purchases, customer_id)


John's Purchases: [{'product_id': 'PROD001', 'name': 'Laptop', 'price': 999.99}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 999.99}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price': 1200.0}, {'product_id': 'PROD001', 'name': 'Laptop', 'price'

/var/folders/45/07b7h57d7d1gx4px8xjqcx640000gn/T/ipykernel_1204/2477979760.py:63: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(self._get_product_customers, product_id)


### Add a customer

In [4]:
CREATE (c:Customer {customer_id: $customer_id, name: $name, email: $email})
RETURN c

SyntaxError: invalid syntax (3759384655.py, line 1)